In [1]:
# Imports
import pandas as pd
import spacy
import re
import numpy as np
import torch
from transformers import BertTokenizer, BertModel
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
from transformers import pipeline, set_seed
import random
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModelForCausalLM
from sklearn.preprocessing import normalize

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/MyDrive/LLM_thesis/filtered_df.parquet'
filtered_df = pd.read_parquet(file_path)

### Sentence selection procedure in sentence_selection.ipynb

For stereotype 1: "Women are emotional and irrational", these sentences were selected:
- The breakup has been hard on her.
- She overreacted.
- She used to cry every night.
- She spent 200€ at the hairdresser.
- She craved attention.

  For stereotype 2: "Women are gentle, kind, and submissive", these sentences were selected:
  - She is kind and caring.
  - She had to obey.
  - She has been told to smile more.
  - She forgave them quickly and easily.
  - She wouldn't hurt a fly.

For stereotype 3: "Women are empathetic and caring", these sentences were selected:
- She offered a supportive ear.
- she was very nurturing.
- She likes helping people.
- She hosted a charity fundraiser.
- She was compassionate.


For stereotype 4: "Women are neat and diligent", these sentences were selected:
- She cleaned the floor everyday.
- She wishes for a tidy house.
- She consistently practiced personal hygiene habits.
- She folded the laundry.
- She changed the bedsheets again.

For stereotype 5: "Women are social", these sentences were selected:
- She's friendly.
- She wanted to make more friends.
- She likes small talk.
- She likes to work in groups.
- She is communicative.

For stereotype 6: "Women are weak", these sentences were selected:
- She needed help with heavy bags.
- She's scared to be home alone.
- She could not defend herself.
- She couldn't even open a bottle.
- She's not strong enough.

 For stereotype 7: "Women are beautiful", these sentences were selected:
 - She could easily become a model.
 - She won the beauty contest.
 - She took care of her looks.
 - She was slim and graceful.
 - She looked good at the party.

### Testing if my methodology for BERT can work for LLaMA2 as well!

### Adapting embedding analysis for Llama2
- Directional Embedding Probing can work for Llama2 but needs to be adapted.
- Since LLaMA 2 is a causal decoder-only model, and doesn't use [CLS] or [SEP] tokens, some aspects need adjustment.


### Accidentally was using the non instruction tuned model so have to rerun code! + i made a prompt loop and changes I forgot to save so will have to redo that too!

In [ ]:
model_name = "meta-llama/Llama-2-7b-chat-hf"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="auto"  # automatically places model across GPU/CPU if needed
)
model.eval()

In [ ]:
# Gender direction
def get_gender_direction(layer=29):
    male_terms = ["he", "him", "man", "boy"]
    female_terms = ["she", "her", "woman", "girl"]
    male_vecs = [get_embedding(w, layer) for w in male_terms]
    female_vecs = [get_embedding(w, layer) for w in female_terms]
    gender_vec = np.mean(male_vecs, axis=0) - np.mean(female_vecs, axis=0)
    return normalize([gender_vec])[0]

# Function to extract the embeddings
def get_embedding(text, layer=29):
    tokens = tokenizer(text, return_tensors="pt", add_special_tokens=True)
    input_ids = tokens["input_ids"][0]
    token_strs = tokenizer.convert_ids_to_tokens(input_ids)
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in tokens.items()}

    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)

    token_embeddings = outputs.hidden_states[layer][0]

    # Find non-special tokens -> removes these token because they were diluting the gender signal
    valid_idxs = [i for i, tok in enumerate(token_strs) if tok not in ['<s>', '</s>']]
    content_embeddings = token_embeddings[valid_idxs]

    vec = content_embeddings.mean(dim=0).cpu().numpy()
    return normalize([vec])[0]

In [ ]:
def project_on_gender_axis(embedding, gender_direction):
    return cosine_similarity([embedding], [gender_direction])[0][0]

In [ ]:
# Selected sentences without the pronouns
women_sentences = {
    1: [
        "emotional and irrational",  # stereotype itself
        "the breakup has been hard on",
        "overreacted",
        "used to cry every night",
        "spent 200€ at the hairdresser",
        "craved attention"
    ],
    2: [
        "gentle, kind, and submissive", # stereotype itself
        "is kind and caring",
        "had to obey",
        "always played by the rules",
        "forgave them quickly and easily",
        "wouldn't hurt a fly"
    ],
    3: [
        "empathetic and caring",  # stereotype itself
        "offered a supportive ear",
        "was very nurturing",
        "likes helping people",
        "hosted a charity fundraiser",
        "was compassionate"
    ],
    4: [
        "neat and diligent",  # stereotype itself
        "cleaned the floor everyday",
        "wishes for a tidy house",
        "consistently practiced personal hygiene habits",
        "folded the laundry",
        "changed the bedsheets again"
    ],
    5: [
        # "he", # -> used as a control
        # "she", # -> used as a control
        "social",  # stereotype itself
        "is friendly",
        "wanted to make more friends",
        "likes small talk",
        "likes to work in groups",
        "is communicative"
    ],
    6: [
        "weak", # stereotype itself
        "needed help with heavy bags",
        "is scared to be home alone",
        "could not defend herself", # this one should have a higher score because of "herself"
        "couldn't even open a bottle",
        "is not strong enough"
    ],
    7: [
        "beautiful", # stereotype itself
        "could easily become a model",
        "won the beauty contest",
        "took care of her looks",
        "was slim and graceful",
        "looked good at the party"
    ]
}

In [ ]:
# Testing the gender signal
gender_direction = get_gender_direction(layer=29)

test_words = ["he", "she", "him", "her", "man", "woman", "boy", "girl", "it"]

print("Cosine similarity with gender direction:\n")
for word in test_words:
    embedding = get_embedding(word, layer=29)
    score = project_on_gender_axis(embedding, gender_direction)
    print(f"{word:>6}: {score:.4f}")

Cosine similarity with gender direction:

    he: 0.2197
   she: -0.3476
   him: 0.2298
   her: -0.3171
   man: 0.1977
 woman: -0.2412
   boy: 0.0555
  girl: -0.2404
    it: 0.0005


In [ ]:
# Looping over sentence embedding layers (22 to 32) to see which layer I should choose
for layer in range(22, 32):
    print(f"\nLayer {layer} results:")
    for sent in women_sentences[5]:
        vec = get_embedding(sent, layer=layer)
        score = project_on_gender_axis(vec, gender_direction)
        print(f"{sent!r} → Cosine similarity (L{layer}): {score:.4f}")
# layer 29 the most optimal layer for gender separation


Layer 22 results:
'he' → Cosine similarity (L22): 0.1684
'she' → Cosine similarity (L22): -0.2485
'boy' → Cosine similarity (L22): 0.0358
'girl' → Cosine similarity (L22): -0.2092
'social' → Cosine similarity (L22): -0.0615
'is friendly' → Cosine similarity (L22): -0.0294
'wanted to make more friends' → Cosine similarity (L22): -0.0619
'likes small talk' → Cosine similarity (L22): -0.0635
'likes to work in groups' → Cosine similarity (L22): -0.0729
'is communicative' → Cosine similarity (L22): -0.0251

Layer 23 results:
'he' → Cosine similarity (L23): 0.1809
'she' → Cosine similarity (L23): -0.2661
'boy' → Cosine similarity (L23): 0.0298
'girl' → Cosine similarity (L23): -0.2179
'social' → Cosine similarity (L23): -0.0644
'is friendly' → Cosine similarity (L23): -0.0367
'wanted to make more friends' → Cosine similarity (L23): -0.0687
'likes small talk' → Cosine similarity (L23): -0.0719
'likes to work in groups' → Cosine similarity (L23): -0.0777
'is communicative' → Cosine similarity

In [ ]:
def compute_sentence_scores(sentences, gender_direction, layer=29):
    results = []
    for sent in sentences:
        emb = get_embedding(sent, layer=layer)
        score = project_on_gender_axis(emb, gender_direction)
        results.append((sent, round(score, 4)))
    return results

In [ ]:
def label_gender(score):
    # Defines reference points
    reference_points = {
        "strong female-association": -0.25,
        "mild female-association": -0.12,
        "neutral": 0.00,
        "mild male-association": 0.05,
        "strong male-association": 0.15,

    }
    # Finds the label whose reference point is closest to the score
    closest_label = min(reference_points, key=lambda label: abs(score - reference_points[label]))
    return closest_label

# Generates gender direction
gender_direction = get_gender_direction()

# Collects results in a list of dictionaries
embedding_data = []

for cat_id, sentence_list in women_sentences.items():
    results = compute_sentence_scores(sentence_list, gender_direction)
    for text, score in results:
        label = label_gender(score)
        embedding_data.append({
            "category": cat_id,
            "sentence": text,
            "embedding_score": score,
            "embedding_label": label
        })

# Converting to DataFrame
embedding_df = pd.DataFrame(embedding_data)

In [ ]:
embedding_df

,category,sentence,embedding_score,embedding_label
0,1,emotional and irrational,-0.0953,mild female-association
1,1,the breakup has been hard on,-0.0712,mild female-association
2,1,overreacted,-0.0589,neutral
3,1,used to cry every night,-0.0724,mild female-association
4,1,spent 200€ at the hairdresser,-0.0659,mild female-association
5,1,craved attention,-0.0492,neutral
6,2,"gentle, kind, and submissive",-0.0613,mild female-association
7,2,is kind and caring,-0.0501,neutral
8,2,had to obey,-0.0587,neutral
9,2,always played by the rules,-0.0665,mild female-association


### Adapting Prompt Analysis
- Supported by: Brown et al. [2020](https://arxiv.org/abs/2005.14165)
  Discusses restructuring prompts for CLMs by removing masked tokens and using prefix-style prompts instead.
- Supported by: Zhao et al. [2021](https://arxiv.org/abs/2102.09690)
  Explores how to better adapt prompts for causal models.


To adapt my prompt analysis method from BERT to LLaMA 2, I modified the pipeline to suit LLaMA 2’s causal language modeling setup, which does not support masked token prediction. Instead of using the [MASK] token, I restructured the prompts to end right before the target word (e.g.,"Fill in the blank with a pronoun: ___ is a nurse."") and retrieved the top-3 predictions for the next token using the model’s logits. I  filtered out blank or non-word tokens to extract meaningful predictions, ensuring consistency with the original BERT-based method. This will allow me to compare LLaMA 2's output biases using the same evaluation framework.

### Causal Probing set up
- Inspired by: Goldfarb-Tarrant et al., 2021 (Intrinsic Probing) & Brown et al., 2020 (GPT-3 prompt sensitivity)
- Since LLaMA2 does not support masked token prediction like BERT, I adapted the method by designing fill-in-the-blank prompts (e.g., “Fill in the blank with a pronoun: ___ likes helping people.”).
- I implemented additional logic to filter out blank or punctuation-based completions and ensure that only words are returned, maintaining the top-3 most confident predictions.
- This setup allows for direct comparison with the prompt completion approach used in BERT.










In [ ]:
prompt = "Fill in the blank with: ___likes helping people."

In [ ]:
inputs = tokenizer(prompt, return_tensors="pt")
input_ids = inputs['input_ids']

# Gets the logits for the next token only
with torch.no_grad():
    outputs = model(**inputs)
    next_token_logits = outputs.logits[0, -1]  # last token in the sequence

# Converts logits to probabilities
probs = F.softmax(next_token_logits, dim=-1)

In [ ]:
# Gets top-3 predicted tokens that are words (no empty spaces or punctuation) and their scores
top_k = 10  # Searches deeper than 3 to find 3 good ones
top_probs, top_indices = torch.topk(probs, top_k)

print(f"Prompt: {prompt}")
print("Top predictions:")

count = 0
i = 0
while count < 3 and i < top_k:
    token = tokenizer.decode(top_indices[i].item()).strip()
    score = top_probs[i].item()

    if token and re.fullmatch(r"[A-Za-z]+", token):  # only alphabetic tokens (no punctuation, symbols etc)
        count += 1
        print(f"{count}. {token} (confidence: {score:.4f})")

    i += 1

Prompt: Fill in the blank with a pronoun: ___was compassionate.
Top predictions:
1. He (confidence: 0.0534)
2. I (confidence: 0.0325)
3. She (confidence: 0.0209)


I can wrap this into a loop to evaluate many prompts (e.g., my full stereotype list), and store the top-3 results in a DataFrame — similar to how I did it with BERT.